In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
train_eassy= pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")

In [ ]:
train_eassy

In [ ]:
train_prompts = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv")

In [ ]:
train_prompts

In [ ]:
df= pd.merge(train_eassy, train_prompts, on='prompt_id')

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df= df.drop(["id"] ,axis=1)
# droped prompt_id and id

In [ ]:
x =df.drop(["generated"],axis=1)

In [ ]:
y= df["generated"]

In [ ]:
y

In [ ]:
x

In [ ]:
x["prompt_name"].unique()

In [ ]:
x

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
columns_to_encode = ['prompt_name', 'instructions','source_text']

le = LabelEncoder()

for column in columns_to_encode:
    x[column] = le.fit_transform(x[column])


In [ ]:
df

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
def remove_stopwords(text):
    words = word_tokenize(text)  
    words_filtered = [word for word in words if word.lower() not in stop_words]
    return ' '.join(words_filtered)

In [ ]:
x['text'] = x['text'].apply(remove_stopwords)

In [ ]:
x

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
import string

In [ ]:
custom_punctuation = string.punctuation + '‘’“”'

In [ ]:
def remove_punctuation(text):
    words = word_tokenize(text)
    words_filtered = [word for word in words if word not in custom_punctuation]
    return ' '.join(words_filtered)

In [ ]:
import string

In [ ]:
x['text'] = x['text'].apply(remove_punctuation)

In [ ]:
x['text'][0]

In [ ]:
def remove_quotes(text):
    
    quotes_to_remove = ['"', "'", '‘', '’', '“', '”',"''",'""',"``","."]
    
    for quote in quotes_to_remove:
        text = text.replace(quote, "")
    return text

In [ ]:
x['text'] = x['text'].apply(remove_quotes)

In [ ]:
x['text'][0]

In [ ]:
def convert_to_lowercase(text):
    return text.lower()

In [ ]:
x['text'] = x['text'].apply(convert_to_lowercase)

In [ ]:
x['text'][0]

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
ps=PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return ' '.join(y)
        

In [ ]:
x['text'] = x['text'].apply(stem)

In [ ]:
x['text'][0]

In [ ]:
x['text'].tail()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
x

In [ ]:
y[704]

In [ ]:
x.loc[704]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(2,7), max_features=10000, stop_words='english', max_df=0.95)


In [ ]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(2, 7), max_features=10000, stop_words='english', max_df=0.95)

# Apply TF-IDF on the 'text' column
tfidf_matrix = tfidf_vectorizer.fit_transform(x['text'])

# Convert the sparse TF-IDF matrix to a dense DataFrame
tfidf_x= pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Reset index of the original DataFrame to align with tfidf_df
x= x.reset_index(drop=True)

# Concatenate the TF-IDF features with the other columns
x = pd.concat([x[['prompt_id', 'prompt_name', 'instructions', 'source_text']], tfidf_x], axis=1)

In [ ]:
x

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.4, random_state=2
                                                )

In [ ]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc= SVC(kernel="linear")

In [ ]:
svc.fit(x_train,y_train)

In [ ]:
y_pred= svc.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy

In [ ]:
print(classification_report(y_test, y_pred))